In [1]:
import pysam
import pandas as pd
import seaborn as sns
from matplotlib import style, colors
import matplotlib.pyplot as plt
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [4]:
vcf = '/projects/da_workspace/szspace/data/giab/HG001_GRCh37_GIAB_highconf_CG-IllFB-IllGATKHC-Ion-10X-SOLID_CHROM1-X_v.3.3.2_highconf_PGandRTGphasetransfer.vcf'


### parse vcf to get SNVs

In [141]:
def identify_category(df, category):
    idx = df.effect.apply(lambda x: [i for i in x.split(';') if category in i][0].split('=')[1])
    categories = df.effect.apply(lambda x: [i for i in x.split(';') if category in i][1].split('=')[1].split(',')) 
#     print(idx, categories)
    df[category] = [y[int(x)-1] for x, y in zip(idx, categories)]
    return df

In [145]:
def parse_vcf(vcf):
    df = pd.read_csv(vcf, comment='#', sep='\t', header=None, low_memory=False)#, nrows=500)
    print(df.head(2))
#     patient = vcf.split('/')[4]
    df = df[[0,1,3,4,7,9]]
    df.columns = ['chr', 'pos', 'ref', 'alt', 'effect', 'genotypetmp']
#     df['platform_num'] = df.effect.apply(lambda x: x.split(';')[0].split('=')[1])
#     df['platform'] = df.effect.apply(lambda x: x.split(';')[1].split('=')[1].split(','))
#     a = df.effect.apply(lambda x: [i for i in x.split(';') if 'platform' in i][0].split('=')[1])
#     b = df.effect.apply(lambda x: [i for i in x.split(';') if 'platform' in i][1].split(',')) 
#     df['platform'] = [y[int(x)-1] for x, y in zip(a, b)]
#     print(z)
    df = identify_category(df, 'platform')
    df = identify_category(df, 'dataset')
#     df = df[(df['effect'].str.contains("platforms=1"))&(df['effect'].str.contains("datasets=1"))]
    if not df.empty:
        df['genotype'] = df['genotypetmp'].apply(lambda x: x.split(':')[0])
        df.drop(['genotypetmp', 'effect'], axis=1, inplace=True)
    df = df[(df.ref.str.len() == 1)&(df.alt.str.len() == 1)]
    return df

In [146]:
df = parse_vcf(vcf)

   0       1            2  3  4   5     6  \
0  1  239339  rs184451216  A  G  50  PASS   
1  1  239482  rs201702841  G  T  50  PASS   

                                                   7  \
0  platforms=1;platformnames=10X;datasets=1;datas...   
1  platforms=2;platformnames=10X,Illumina;dataset...   

                              8                               9  
0  GT:DP:ADALL:AD:GQ:IGT:IPS:PS        0/1:0:0,0:0,0:99:0/1:.:.  
1  GT:DP:ADALL:AD:GQ:IGT:IPS:PS  0/1:403:92,104:0,0:198:0/1:.:.  


In [151]:
df.shape
df.head(3)

(3257176, 7)

,chr,pos,ref,alt,platform,dataset,genotype
0,1,239339,A,G,10X,10XChromium,0/1
1,1,239482,G,T,Illumina,HiSeqPE300x,0/1
2,1,240147,C,T,Illumina,HiSeqPE300x,0/1


In [153]:
of = '/projects/da_workspace/szspace/data/giab/ground_truth_calls.txt'
df.to_csv(of,index=False, sep='\t')

In [149]:
df[(df.platform == 'Illumina')&(df.dataset == 'HiSeqPE300x')].shape

(13438, 7)